In [ ]:
!pip install -q langchain==0.0.150 pypdf pandas matplotlib tiktoken textract transformers openai faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.4/648.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 96.3 MB/s eta 0:00:0

In [ ]:
#os.environ["OPENAI_API_KEY"] = 'api key kita'

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from transformers import GPT2TokenizerFast
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain

In [ ]:
file = 'file_path'
loader = PyPDFLoader(file)
pages = loader.load_and_split()
print(pages[0])

chunks = pages

page_content="DEA ALIFIA MAHARANI SIREGAR\n085959865989 | deaalifiaa@gmail.com\nWest Jakarta\nUndergraduate Mathematics Student at Universitas Indonesia. Interested and looking for opportunities in data analysis, machine learning, \nand data science. Actively participate in projects and events related to data science.\nEducation\nUniversitas Indonesia - Depok, Indonesia\nUndergraduate in Mathematics, 3.62/4.00\nRelevant Courses: Introduction to Data Science, Data Science, Algorithm and Programming, Data Structure, Mathematical \nProgramming, Linear Algebra, Mathematical Modelling, Computational Intelligence\nParticipated in more than 15 campus events, such as webinars and workshops, as well as in projects, committees, and organizations.\nSMAN 78 - Jakarta, Indonesia Jul 2017 - Jul 2020\nWork & Organization Experiences\nLOGIKA UI 2023 Jul 2022 - Feb 2023\nMarketing Coordinator\nNational Mathematics Competition\nManaged to get 1000+ participants by creating ads based on research on previ

In [ ]:
import textract
doc = textract.process(file)

with open(file, 'w') as f:
    f.write(doc.decode('utf-8'))

with open(file, 'r') as f:
    text = f.read()

tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 512,
    chunk_overlap  = 24,
    length_function = count_tokens,
)

chunks = text_splitter.create_documents([text])

In [ ]:
token_counts = [count_tokens(chunk.page_content) for chunk in chunks]
df = pd.DataFrame({'Token Count': token_counts})

In [ ]:
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(chunks, embeddings)

In [ ]:
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")

def context_interview(query):
  docs = db.similarity_search(query)
  return chain.run(input_documents=docs, question=query)

In [ ]:
skills = 'what are the skills contained in the document?'
experiences = 'what are the experiences contained in the document?'

In [ ]:
context_interview(skills)